## Import important Libraries

In [1]:
import numpy as np
import time
import cv2 as cv
import os
import glob
import matplotlib.pyplot as plt

## YOLO Weights and Config

In [2]:
weights_path = os.path.join("yolo", "yolov3.weights")
config_path = os.path.join("yolo", "yolov3.cfg")

print ("Loaded Yolo weights and config")

Loaded Yolo weights and config


## Load Neural Network in OpenCV

In [3]:
net = cv.dnn.readNetFromDarknet(config_path, weights_path)

In [4]:
names = net.getLayerNames()

In [5]:
def pass_yolov3(img,layers):
    blob = cv.dnn.blobFromImage(img, 1/255.0, (416,416), crop=False, swapRB= False)

    net.setInput(blob)
    start_t = time.time()

    layers_output = net.forward(layers)
    #print("A forward pass through yolov3 took {}".format(time.time()-start_t))
    return layers_output

In [6]:
def get_objectplace(layers_output,W,H):
    boxes = []
    confidences =[]
    classIDs = []
    for output in layers_output:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            if confidence > 0.80:
                box = detection[:4] * np.array([W, H, W, H])
                bx, by, bw, bh = box.astype("int")

                x = int(bx-(bw/2))
                y = int(by - (bh/2))

                boxes.append([x,y,int(bw), int(bh)])
                confidences.append(float(confidence))
                classIDs.append(classID) 
    return boxes , confidences,classIDs

In [7]:
def draw_boxes(img,boxes,confidences,classIDs):
    idxs = cv.dnn.NMSBoxes(boxes, confidences, 0.8, 0.6)
    labels_path = os.path.join("yolo","coco.names")
    labels = open(labels_path).read().strip().split("\n")

    for i in idxs.flatten():
        (x,y) = [boxes[i][0], boxes[i][1]]
        (w,h) = [boxes[i][2], boxes[i][3]]
        cv.rectangle(img, (x,y), (x+w, y+h), (0,255,255), 2)
        cv.putText(img,"{}: {}".format(labels[classIDs[i]], round(confidences[i],3)), (x,y-5), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,139,139),2)
    return img    

In [8]:
# Reading an image
images_paths = glob.glob("images/*.jpg")
images = [cv.imread(image) for image in images_paths]

# getting img dimensions and printing it
image_idx = 0

for test_img in images:
    img = cv.cvtColor(test_img, cv.COLOR_BGR2RGB)
    (H, W) = img.shape[:2]
    layers_names = [names[i[0]-1] for i in net.getUnconnectedOutLayers()]
    layers_output= pass_yolov3(img, layers_names)
    boxes , confidences,classIDs= get_objectplace(layers_output,W,H)
    if len(boxes)>0:
        detected_obj_img=draw_boxes(img,boxes,confidences,classIDs)
    else:
        detected_obj_img=img
    detected_obj_img = cv.cvtColor(detected_obj_img, cv.COLOR_BGR2RGB)
    cv.imwrite('output/test_cars_detection{}.jpg'.format(str(image_idx)), detected_obj_img)
    image_idx = image_idx + 1

In [10]:
video_name='project_video.mp4'
cap = cv.VideoCapture(video_name)
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

out = cv.VideoCapture(0)
fps = int(cap.get(cv.CAP_PROP_FPS))
fourcc = cv.VideoWriter_fourcc(*'mp4v')
out = cv.VideoWriter('output/car_detection_{}'.format(video_name),fourcc, fps, ( 1280, 720 ))

while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    # Display the resulting frame
    frame = cv.resize(frame, ( 1280, 720 ))
    frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    (H, W) = frame.shape[:2]
    layers_names = [names[i[0]-1] for i in net.getUnconnectedOutLayers()]
    layers_output= pass_yolov3(frame,layers_names)
    boxes , confidences,classIDs= get_objectplace(layers_output,W,H)
    if len(boxes)>0:
        detected_obj_img=draw_boxes(frame,boxes,confidences,classIDs)
    else:
        detected_obj_img=frame
    detected_obj_img = cv.cvtColor(detected_obj_img, cv.COLOR_BGR2RGB)
    
    out.write(detected_obj_img)
    cv.imshow('Frame',detected_obj_img)
    # Press Q on keyboard to  exit
    if cv.waitKey(25) & 0xFF == ord('q'):
      break

  else: 
    break

# When everything done, release the video capture object
cap.release()
out.release()

# Closes all the frames
cv.destroyAllWindows()